# H and M Create Embeddings

In [ ]:
import numpy as np
import pandas as pd 
from sentence_transformers import SentenceTransformer
from pathlib import Path
import time
from collections import Counter
import requests
from io import StringIO
from typing import List, Tuple
import requests
from io import BytesIO
import os
from PIL import Image
import matplotlib.pyplot as plt

def create_enhanced_descriptions(df):
    """
    Create enhanced product descriptions with grouped attributes.
    Returns DataFrame with unique products and their enhanced descriptions.
    """
    print("\nGrouping products and creating enhanced descriptions...")
    
    # Group by product code
    product_groups = df.groupby('product_code').agg({
        'prod_name': 'first',
        'product_type_name': 'first',
        'detail_desc': 'first',
        'colour_group_name': lambda x: sorted(list(set(x))),
        'graphical_appearance_name': lambda x: sorted(list(set(x))),
        'article_id': list  # Keep all article IDs for this product
    }).reset_index()
    
    # Create enhanced descriptions with simple progress tracking
    results = []
    total_products = len(product_groups)
    print(f"Processing {total_products} products...")
    
    for idx, row in product_groups.iterrows():
        if idx % 1000 == 0:  # Print progress every 1000 items
            print(f"Progress: {idx}/{total_products} products processed ({(idx/total_products*100):.1f}%)")
            
        colors = f"[Colors: {', '.join(row['colour_group_name'])}]"
        patterns = f"[Patterns: {', '.join(row['graphical_appearance_name'])}]"
        
        enhanced_desc = (f"{row['prod_name']} | {row['product_type_name']} | "
                        f"{colors} {patterns} | {row['detail_desc']}")
        
        results.append({
            'product_code': row['product_code'],
            'product_name': row['prod_name'],
            'article_ids': row['article_id'],
            'num_variants': len(row['colour_group_name']),
            'colors': row['colour_group_name'],
            'patterns': row['graphical_appearance_name'],
            'embedding_string': enhanced_desc
        })
    
    print(f"Completed processing all {total_products} products")
    return pd.DataFrame(results)

def batch_encode(model, texts, batch_size=32):
    """
    Encode texts in batches with simple progress tracking.
    """
    embeddings = []
    total_batches = (len(texts) + batch_size - 1) // batch_size
    print(f"\nCreating embeddings for {len(texts)} texts in {total_batches} batches...")
    start_time = time.time()
    
    for i in range(0, len(texts), batch_size):
        batch_num = i // batch_size + 1
        if batch_num % 10 == 0:  # Print progress every 10 batches
            elapsed = time.time() - start_time
            avg_time_per_batch = elapsed / batch_num
            remaining_batches = total_batches - batch_num
            est_remaining = remaining_batches * avg_time_per_batch
            
            print(f"Batch {batch_num}/{total_batches} "
                  f"({(batch_num/total_batches*100):.1f}%) - "
                  f"Est. remaining time: {est_remaining/60:.1f} minutes")
        
        batch = texts[i:i + batch_size]
        batch_embeddings = model.encode(batch, show_progress_bar=False)
        embeddings.append(batch_embeddings)
    
    print("\nEmbedding creation completed!")
    return np.vstack(embeddings)
    
def print_dataset_statistics(df, enhanced_df):
    """Print comprehensive statistics about the dataset and embeddings."""
    print("\n" + "="*50)
    print("DATASET STATISTICS")
    print("="*50)
    
    print("\nBasic Statistics:")
    print(f"Total number of articles: {len(df)}")
    print(f"Unique products: {len(enhanced_df)}")
    print(f"Average variants per product: {enhanced_df['num_variants'].mean():.2f}")
    
    print("\nVariant Distribution:")
    variant_counts = enhanced_df['num_variants'].value_counts().sort_index()
    for variants, count in variant_counts.items():
        print(f"Products with {variants} variants: {count}")
    
    print("\nTop 10 Products with Most Variants:")
    most_variants = enhanced_df.nlargest(10, 'num_variants')
    for _, row in most_variants.iterrows():
        print(f"{row['product_name']}: {row['num_variants']} variants")
    
    # Analyze colors
    all_colors = [color for colors in enhanced_df['colors'] for color in colors]
    print("\nTop 10 Most Common Colors:")
    for color, count in Counter(all_colors).most_common(10):
        print(f"{color}: {count} products")
    
    # Analyze patterns
    all_patterns = [pattern for patterns in enhanced_df['patterns'] for pattern in patterns]
    print("\nPattern Distribution:")
    for pattern, count in Counter(all_patterns).most_common():
        print(f"{pattern}: {count} products")
    
    # Analyze string lengths
    string_lengths = enhanced_df['embedding_string'].str.len()
    print("\nEmbedding String Length Statistics:")
    print(f"Average length: {string_lengths.mean():.1f} characters")
    print(f"Maximum length: {string_lengths.max()} characters")
    print(f"Minimum length: {string_lengths.min()} characters")


def save_embeddings(embeddings, product_data, filepath):
    """
    Save embeddings and related data as a compressed NPZ file.
    Handles lists of different lengths properly.
    """
    if not filepath.endswith('.npz'):
        filepath = f"{filepath}.npz"
    
    # Convert article_ids to a string representation for each product
    article_ids_str = [','.join(map(str, ids)) for ids in product_data['article_ids']]
    
    # Prepare metadata
    metadata = {
        'num_products': len(product_data),
        'embedding_dim': embeddings.shape[1],
        'date_created': np.datetime64('now').astype(str),
        'model_used': 'all-MiniLM-L6-v2',
        'description': 'Product embeddings with grouped color variants'
    }
    
    # Save compressed npz file
    np.savez_compressed(
        filepath,
        embeddings=embeddings,
        product_codes=product_data['product_code'].values,
        article_ids_str=article_ids_str,  # Store as comma-separated strings
        product_names=product_data['product_name'].values,
        embedding_strings=product_data['embedding_string'].values,
        metadata=metadata
    )
    
    # Print file info
    file_size_mb = Path(filepath).stat().st_size / (1024 * 1024)
    print(f"\nSaved embeddings file:")
    print(f"Path: {filepath}")
    print(f"Size: {file_size_mb:.2f} MB")
    print(f"Products: {len(product_data)}")
    print(f"Embedding dimensions: {embeddings.shape[1]}")
    
    # Print first few products as examples
    print("\nFirst few products saved:")
    for i in range(min(3, len(product_data))):
        print(f"\nProduct: {product_data['product_name'].iloc[i]}")
        print(f"Article IDs: {article_ids_str[i]}")
        print(f"Embedding string length: {len(product_data['embedding_string'].iloc[i])}")

def create_embeddings():
    
    start_time = time.time()
 
    url = "https://github.com/calvinw/semantic-search/raw/refs/heads/main/articles.csv"
    df = pd.read_csv(url)

    enhanced_df = create_enhanced_descriptions(df)

    # Add this section to print example embedding strings
    print("\nExample embedding strings that will be embedded:")
    print("\n" + "="*80)
    for i in range(5):  # Show first 5 examples
        print(f"\nExample {i+1}:")
        print(enhanced_df['embedding_string'].iloc[i])
        print("-"*80)

    print_dataset_statistics(df, enhanced_df)
    
    print("\nInitializing embedding model...")
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Create embeddings with custom batching and progress tracking
    embeddings = batch_encode(
        model, 
        enhanced_df['embedding_string'].tolist(),
        batch_size=32
    )
    
    save_embeddings(embeddings, enhanced_df, "product_embeddings")
    
    elapsed_time = time.time() - start_time
    print(f"\nTotal processing time: {elapsed_time:.1f} seconds")

In [ ]:
create_embeddings()

## Creating the Embedding Strings

### Group the Articles with Same Product Code

First, group the data by `product_code` using `groupby()`. This is
because a single product (like a t-shirt) might have multiple variants
(different colors/patterns) in the dataset. During this grouping, it:

-   Takes the first occurrence of `prod_name`, `product_type_name`, and
    `detail_desc` for each product
-   Creates sorted lists of unique colors (`colour_group_name`) and
    patterns (`graphical_appearance_name`)
-   Keeps a list of all `article_id`s associated with each product

### Create Enhanced Description Containing Variants (Color, Pattern)

Then, for each product group, create an enhanced description by
combining:

-   Product name
-   Product type
-   A list of all available colors
-   A list of all available patterns
-   The detailed description

### Strap Top Example

For example “Strap top” (product code “0108775”) is combined like this:

-   Combine all colors: \[“Black”, “White”, “Off White”\]
-   Combine all patterns: \[“Solid”, “Stripe”\]
-   Create an enhanced description like: “Strap top \| Vest top \|
    \[Colors: Black, Off White, White\] \[Patterns: Solid, Stripe\] \|
    Jersey top with narrow shoulder straps.”

### The Enhanced Data Frame

The final enhanced DataFrame contains:

-   product_code: Unique identifier for each product
-   product_name: Name of the product
-   article_ids: List of all variant IDs for this product
-   num_variants: Number of color variants available
-   colors: List of available colors
-   patterns: List of available patterns
-   embedding_string: The enhanced description combining all attributes

This enhanced description could be useful for:

-   Search functionality that considers all variants
-   Product recommendations based on similar attributes
-   Better product discovery by including all available options in the
    description

### Strap Top Example Details

Here is a complete example using the “Strap top” (product code 0108775)
from the data, showing both the data before and after

#### BEFORE (Separate Article Rows in Original Dataframe df)

| article_id | product_code | prod_name     | product_type_name | detail_desc                             | colour_group_name | graphical_appearance_name |
|--------|---------|--------|------------|---------|------------|----------------|
| 0108775015 | 0108775      | Strap top     | Vest top          | Jersey top with narrow shoulder straps. | Black             | Solid                     |
| 0108775044 | 0108775      | Strap top     | Vest top          | Jersey top with narrow shoulder straps. | White             | Solid                     |
| 0108775051 | 0108775      | Strap top (1) | Vest top          | Jersey top with narrow shoulder straps. | Off White         | Stripe                    |

#### AFTER (Single Product Row in the Enhanced DataFrame)

| Field            | Value                                                                                                                               |
|----------------------------------|--------------------------------------|
| product_code     | 0108775                                                                                                                             |
| product_name     | Strap top                                                                                                                           |
| article_ids      | \[‘0108775015’, ‘0108775044’, ‘0108775051’\]                                                                                        |
| num_variants     | 3                                                                                                                                   |
| colors           | \[‘Black’, ‘Off White’, ‘White’\]                                                                                                   |
| patterns         | \[‘Solid’, ‘Stripe’\]                                                                                                               |
| embedding_string | Strap top \| Vest top \| \[Colors: Black, Off White, White\] \[Patterns: Solid, Stripe\] \| Jersey top with narrow shoulder straps. |

The transformation:

1.  Takes 3 separate variant-level rows
2.  Combines them into 1 product-level row
3.  Preserves all variant information in organized lists
4.  Creates a comprehensive description string that includes all color
    and pattern options

### More Examples:

#### Example 1 - FLEECE PYJAMA:

**BEFORE**

| article_id | product_code | prod_name     | product_type_name        | detail_desc                 | colour_group_name | graphical_appearance_name |
|--------|---------|--------|------------|---------|------------|----------------|
| 0174057022 | 0174057      | FLEECE PYJAMA | Pyjama jumpsuit/playsuit | All-in-one pyjamas in soft… | Light Pink        | All over pattern          |
| 0174057026 | 0174057      | FLEECE PYJAMA | Pyjama jumpsuit/playsuit | All-in-one pyjamas in soft… | Light Grey        | All over pattern          |
| 0174057029 | 0174057      | FLEECE PYJAMA | Pyjama jumpsuit/playsuit | All-in-one pyjamas in soft… | White             | Dot                       |
| 0174057030 | 0174057      | FLEECE PYJAMA | Pyjama jumpsuit/playsuit | All-in-one pyjamas in soft… | Light Turquoise   | Stripe                    |

**AFTER**

| Field            | Value                                                                                                                                                                                                                                                                            |
|----------------------------------|--------------------------------------|
| product_code     | 0174057                                                                                                                                                                                                                                                                          |
| product_name     | FLEECE PYJAMA                                                                                                                                                                                                                                                                    |
| article_ids      | \[‘0174057022’, ‘0174057026’, ‘0174057029’, ‘0174057030’\]                                                                                                                                                                                                                       |
| num_variants     | 4                                                                                                                                                                                                                                                                                |
| colors           | \[‘Light Grey’, ‘Light Pink’, ‘Light Turquoise’, ‘White’\]                                                                                                                                                                                                                       |
| patterns         | \[‘All over pattern’, ‘Dot’, ‘Stripe’\]                                                                                                                                                                                                                                          |
| embedding_string | FLEECE PYJAMA \| Pyjama jumpsuit/playsuit \| \[Colors: Light Grey, Light Pink, Light Turquoise, White\] \[Patterns: All over pattern, Dot, Stripe\] \| All-in-one pyjamas in soft, patterned fleece that fasten down the front and along one leg. Ribbing at the cuffs and hems. |

#### Example 2 - Jerry jogger bottoms:

**BEFORE**

| article_id | product_code | prod_name            | product_type_name | detail_desc              | colour_group_name | graphical_appearance_name |
|--------|---------|--------|------------|---------|------------|----------------|
| 0118458003 | 0118458      | Jerry jogger bottoms | Trousers          | Trousers in sweatshirt … | Dark Grey         | Melange                   |
| 0118458028 | 0118458      | Jerry jogger bottoms | Trousers          | Trousers in sweatshirt … | Black             | Solid                     |
| 0118458034 | 0118458      | Jerry jogger bottoms | Trousers          | Trousers in sweatshirt … | Pink              | Solid                     |
| 0118458039 | 0118458      | Jerry jogger bottoms | Trousers          | Trousers in sweatshirt … | Dark Blue         | Melange                   |

**AFTER**

| Field            | Value                                                                                                                                                                                                                                                   |
|----------------------------------|--------------------------------------|
| product_code     | 0118458                                                                                                                                                                                                                                                 |
| product_name     | Jerry jogger bottoms                                                                                                                                                                                                                                    |
| article_ids      | \[‘0118458003’, ‘0118458028’, ‘0118458034’, ‘0118458039’\]                                                                                                                                                                                              |
| num_variants     | 4                                                                                                                                                                                                                                                       |
| colors           | \[‘Black’, ‘Dark Blue’, ‘Dark Grey’, ‘Pink’\]                                                                                                                                                                                                           |
| patterns         | \[‘Melange’, ‘Solid’\]                                                                                                                                                                                                                                  |
| embedding_string | Jerry jogger bottoms \| Trousers \| \[Colors: Black, Dark Blue, Dark Grey, Pink\] \[Patterns: Melange, Solid\] \| Trousers in sweatshirt fabric with an elasticated drawstring waist, side pockets, a back pocket and ribbed hems. Soft brushed inside. |

In both examples, you can see how the transformation:

1.  Combines multiple variants of the same product into a single row
2.  Aggregates all colors and patterns into sorted, unique lists
3.  Preserves all article IDs for reference
4.  Creates a comprehensive description that includes all available
    options
5.  Maintains the core product information while consolidating the
    variant-specific details

### Why We Are Doing This

This makes it much easier to see how multiple product variants are
consolidated into a single, information-rich record while maintaining
all the important details from the original data.

The grouping reduces multiple rows of variant-level information into a
single row of product-level information while preserving all the
important variant details in organized lists.